# Merge places

Join the buildings to places defined by the U.S. Census Bureau.

In [1]:
import os
import glob
import geopandas as gpd

In [2]:
%store -r

In [5]:
places = gpd.read_file(os.path.join(input_dir, "places", "tl_2018_06_place.shp")).to_crs({'init': 'epsg:4326'})

In [6]:
places.crs

{'init': 'epsg:4326'}

In [7]:
len(places)

1521

In [39]:
def sjoin_batch(points_path, verbose=False):
    """
    Reads in a batch of buildings and returns them with a column that indicates if they fall within a Census designated place.
    """
    # Set the path
    number = points_path.split("-batch-")[1].replace(".shp", "")
    outpath = os.path.join(output_dir, f"california-building-points-batch-{number}-to-places.csv")
    
    # Skip if it already exists
    if os.path.exists(outpath):
        if verbose:
            print(f"Skipping batch {number}")
        return
        
    # Read in the file
    points = gpd.read_file(points_path)
    
    # Run a spatial join against the fire zones
    sjoin = gpd.sjoin(points, places, how="left", op="intersects")
    
    if verbose:
        print(f"{len(sjoin[~gpd.pd.isnull(sjoin.NAME)])}/{len(sjoin)} from {points_path} in a CDP")
    
    # Save the file
    if verbose:
        print(f"Creating {outpath}")
    trimmed = sjoin[[
        'FID',
        'PLACEFP',
        'NAMELSAD',
    ]]
    trimmed.to_csv(outpath, index=False)

In [40]:
path_list = sorted(
    glob.glob(os.path.join(output_dir, "california-building-points-batch-*.shp")),
    key=lambda x: int(x.split("-batch-")[1].replace(".shp", ""))
)

In [41]:
len(path_list)

1099

In [42]:
sjoin_batch(path_list[0], verbose=True)

Skipping batch 1


In [43]:
for p in path_list: sjoin_batch(p)

In [44]:
csv_list = glob.glob(os.path.join(output_dir, "california-building-points-batch-*-to-places.csv"))

In [46]:
csv_df = gpd.pd.concat([gpd.pd.read_csv(f) for f in csv_list])

In [48]:
len(csv_df)

10988525

In [49]:
csv_df.to_csv(os.path.join(output_dir, "california-building-points-to-places.csv"), index=False)